In [42]:
import numpy as np
from PIL import Image, ImageDraw
import json
import matplotlib.pyplot as plt 
import cv2
from jsonmerge import merge
from tifffile import imsave
import os
from os.path import splitext
from glob import glob

In [43]:
folder_lst = ['Fifth_3',
              'First_10',
              'Forth_6',
              'Second_0',
              'Second_2',
              'Second_4',
              'Sixth_2',
              'Sixth_3']

# folder_lst = ['Fifth_5',
#               'Sixth_1']

# folder_lst = ['Fifth_2',
#               'Forth_3',
#               'Second_7',
#               'Second_9',
#               'Sixth_9']


In [45]:
for folder_name in folder_lst:
    dir_ = '/home/natasha/unet4/axial_data/train/' + folder_name + '/'
    json1 = 'T2Tra_' + folder_name + '_s.json'
    json2 = 'T2Tra_' + folder_name + '_h.json'
    
    with open(dir_ + json1) as file:
        j1 = json.load(file)
        
    with open(dir_ + json2) as file:
        j2 = json.load(file)
    
    raw_data = merge(j1, j2)
    all_keys = []

    for key in raw_data['_via_img_metadata'].keys():
        all_keys.append(key)

    pics_path = dir_ + 'pictures/'
    masks_path = dir_ + 'masks/'

    if not os.path.exists(masks_path):
        os.makedirs(masks_path)

    fucked_imgs = []
    zero_files = []
    not_found = []
    for k in range(0, len(all_keys)):
        filename = raw_data['_via_img_metadata'][all_keys[k]]['filename']

        try:
            image = Image.open(pics_path + filename)
            width = image.size[0] 
            height = image.size[1]

            img1 = Image.new('L',(width, height), 0)
            img2 = Image.new('L',(width, height), 0)
            img3 = Image.new('L',(width, height), 0)
            img4 = Image.new('L',(width, height), 0)    
            img5 = Image.new('L',(width, height), 0)
            img6 = Image.new('L',(width, height), 0)
            for i in range(len(raw_data['_via_img_metadata'][all_keys[k]]['regions'])):
                try:
                    xs = raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['shape_attributes']['all_points_x']
                    ys = raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['shape_attributes']['all_points_y']
                    all_pairs = []
                    for j in range(len(ys)):
                        all_pairs.append((xs[j], ys[j]))

                    if raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['A3'] == 'disc':
                        ImageDraw.Draw(img1).polygon(all_pairs, outline=1, fill=1)

                    if raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['A3'] == 'spinal canal':
                        ImageDraw.Draw(img2).polygon(all_pairs, outline=1, fill=1) 
                        
                    if raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['A3'] == 'nerv':
                        ImageDraw.Draw(img3).polygon(all_pairs, outline=1, fill=1)
                        
                    if raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['A3'] == 'neural foramen':
                        ImageDraw.Draw(img4).polygon(all_pairs, outline=1, fill=1)
                        
                    if raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['A3'] == 'size of spinal canal':
                        ImageDraw.Draw(img5).polygon(all_pairs, outline=1, fill=1)

                    if raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['A3'] == 'size of herniation':
                        ImageDraw.Draw(img6).polygon(all_pairs, outline=1, fill=1)


                except:
                    fucked_imgs.append(filename)
                    
            mask1 = np.array(img1)
            mask2 = np.array(img2)
            mask3 = np.array(img3)
            mask4 = np.array(img4)
            mask5 = np.array(img5)
            mask6 = np.array(img6)

#             mask = mask1 + 2 * mask2 + 3 * mask3 + 4 * mask4 + 5 * mask5 + 6 * mask6
            mask = np.dstack((mask1, mask2, mask3, mask4, mask5, mask6))
            if len(np.unique(mask)) == 1:
                zero_files.append(filename)

            
        except:
            not_found.append(filename)
        

        if filename not in not_found:
            if filename not in fucked_imgs:
                if filename not in zero_files:
                    if len(np.unique(mask)) > 1:
                        filename = filename[:-3] + 'npy'
#                         print(filename)
#                         print(np.unique(mask))
                        np.save(masks_path + filename, mask)

In [41]:
len(zero_files), len(fucked_imgs), len(not_found)

(82, 82, 0)

In [42]:
not_found

['00088.PNG',
 '00092.PNG',
 '00126.PNG',
 '0157.PNG',
 '00158.PNG',
 '00160.PNG',
 '00201.PNG',
 '00209.PNG',
 '00223.PNG',
 '00224.PNG',
 '00227.PNG',
 '00228.PNG',
 '00241.PNG',
 '0256.PNG',
 '00257.PNG',
 '00261.PNG',
 '00265.PNG',
 '0269.PNG',
 '00272.PNG',
 '00274.PNG',
 '00279.PNG',
 '00292.PNG',
 '00295.PNG',
 '00302.PNG',
 '00303.PNG',
 '0678.PNG',
 '00784.PNG',
 '00789.PNG',
 '00797.PNG',
 '0841.PNG',
 '0845.PNG',
 '0905.PNG',
 '00907.PNG',
 '0913.PNG',
 '0935.PNG',
 '00936.PNG',
 '00944.PNG',
 '00982.PNG',
 '00987.PNG',
 '0987.PNG',
 '00992.PNG',
 '01000.PNG',
 '01005.PNG',
 '01018.PNG',
 '1023.PNG',
 '01029.PNG',
 '01036.PNG',
 '1045.PNG',
 '01050.PNG',
 '01055.PNG',
 '01058.PNG',
 '1066.PNG',
 '1078.PNG']

In [15]:
len(broken_imgs)

71

In [46]:
filename = filename[:-3] + 'npy'
np.save(masks_path + filename, mask)

In [31]:
img6.show()

In [7]:
print(f'folder_name = {folder_name}')
print(f'# not present in folder from json = {len(zero_files)}')
print(f'# imgs in json = {len(all_keys)}')
print(f'not present files = {zero_files}')

folder_name = Second_2
# not present in folder from json = 44
# imgs in json = 312
not present files = ['00268.PNG', '00269.PNG', '0269.PNG', '00272.PNG', '00273.PNG', '00274.PNG', '00276.PNG', '0276.PNG', '00282.PNG', '0283.PNG', '00285.PNG', '00286.PNG', '00287.PNG', '0287.PNG', '00289.PNG', '00292.PNG', '0292.PNG', '00293.PNG', '00295.PNG', '00297.PNG', '0297.PNG', '00298.PNG', '00300.PNG', '0300.PNG', '00301.PNG', '00302.PNG', '00303.PNG', '00304.PNG', '00324.PNG', '00327.PNG', '0327.PNG', '00329.PNG', '00332.PNG', '0332.PNG', '00334.PNG', '00335.PNG', '00362.PNG', '0362.PNG', '00363.PNG', '00365.PNG', '00366.PNG', '00367.PNG', '0367.PNG', '00369.PNG']


In [23]:
imgs_dir = '/home/natasha/unet4/axial_data/train/Fifth_3/pictures/'
masks_dir = '/home/natasha/unet4/axial_data/train/Fifth_3/masks/'
ids = [splitext(file)[0] for file in os.listdir(imgs_dir) if not file.startswith('.')]

In [27]:
imgs_dir = '/home/natasha/unet4/axial_data/train/'
folders = os.listdir(imgs_dir)
mask_paths = []
img_paths = []
for i, folder in enumerate(folders):
    path = os.path.join(imgs_dir, folder, 'pictures')
    ids = [splitext(file)[0] for file in os.listdir(path) if not file.startswith('.')]
#     print(f'path = {path}')
    for idx in ids:
        mask_paths.append(os.path.join(imgs_dir, folder, 'masks', str(idx)) + '.npy')
        img_paths.append(os.path.join(imgs_dir, folder, 'pictures', str(idx)) + '.PNG')
    
for i, _ in enumerate(mask_paths):
    print(f'{mask_paths[i]}{img_paths[i]}\n')




/home/natasha/unet4/axial_data/train/Sixth_2/masks/00211.npy/home/natasha/unet4/axial_data/train/Sixth_2/pictures/00211.PNG

/home/natasha/unet4/axial_data/train/Sixth_2/masks/0287.npy/home/natasha/unet4/axial_data/train/Sixth_2/pictures/0287.PNG

/home/natasha/unet4/axial_data/train/Sixth_2/masks/01225.npy/home/natasha/unet4/axial_data/train/Sixth_2/pictures/01225.PNG

/home/natasha/unet4/axial_data/train/Sixth_2/masks/0354.npy/home/natasha/unet4/axial_data/train/Sixth_2/pictures/0354.PNG

/home/natasha/unet4/axial_data/train/Sixth_2/masks/00437.npy/home/natasha/unet4/axial_data/train/Sixth_2/pictures/00437.PNG

/home/natasha/unet4/axial_data/train/Sixth_2/masks/1098.npy/home/natasha/unet4/axial_data/train/Sixth_2/pictures/1098.PNG

/home/natasha/unet4/axial_data/train/Sixth_2/masks/0154.npy/home/natasha/unet4/axial_data/train/Sixth_2/pictures/0154.PNG

/home/natasha/unet4/axial_data/train/Sixth_2/masks/0348.npy/home/natasha/unet4/axial_data/train/Sixth_2/pictures/0348.PNG

/home/nata

In [20]:
for path_id in path_ids:
    print(path_id[0], path_id[1])
    mask_file = glob(os.path.join(path_id[0], 'masks', f'{path_id[1]}.npy'))[0]
    img_file = glob(os.path.join(path_id[0], 'pictures', f'{path_id[1]}.PNG'))[0]

/home/natasha/unet4/axial_data/train/Sixth_2 ['00211', '0287', '01225', '0354', '00437', '1098', '0154', '0348', '01095', '01138', '1271', '01233', '00428', '00365', '00350', '1346', '00387', '00247', '00422', '1251', '01274', '00439', '1256', '01289', '0380', '00322', '1300', '01271', '0322', '0395', '1343', '00314', '01107', '01123', '1334', '0390', '01238', '00153', '00246', '0428', '00392', '1132', '00417', '01321', '1186', '01317', '00434', '00405', '00353', '01335', '1204', '1207', '01268', '01246', '1127', '01200', '1143', '1233', '01192', '1108', '00406', '01214', '1153', '1118', '00155', '01340', '00312', '0433', '00403', '00440', '00420', '01219', '01131', '1248', '00400', '1223', '1313', '1091', '01355', '0438', '0398', '01241', '00425', '01159', '01187', '00285', '1276', '0367', '1354', '1179', '1243', '00248', '00320', '0345', '0351', '01197', '00127', '01141', '1169', '00367', '00363', '01117', '1290', '0385', '00360', '00401', '00287', '01103', '01179', '00332', '01120',

IndexError: list index out of range

In [25]:
splitext(file)[0]

TypeError: expected str, bytes or os.PathLike object, not _io.TextIOWrapper

In [24]:
print(ids)

['00437', '00428', '00460', '00422', '00439', '00409', '0480', '0395', '00490', '00497', '00448', '0428', '00392', '00417', '00506', '00434', '00405', '00415', '0510', '00451', '00444', '0487', '0494', '0515', '00406', '0469', '0433', '00403', '00440', '00420', '0501', '00486', '00400', '0438', '0398', '00425', '00459', '00484', '00476', '00408', '00401', '00471', '00457', '00498', '0490', '00424', '00481', '00505', '0410', '00499', '0406', '00456', '0418', '00503', '0453', '00475', '00487', '00414', '0476', '0409', '00429', '00502', '00466', '00480', '00513', '00431', '00473', '00514', '0443', '00465', '00419', '0506', '00463', '00494', '00421', '0472', '00443', '0450', '00510', '00500', '00412', '0423', '0464', '00492', '00449', '00507', '0458', '00485', '00493', '0415', '00395', '00413', '00454', '00495', '0401', '00515', '00396', '00452', '00511', '00468']
